In [1]:
import cv2
from darkflow.net.build import TFNet
import numpy as np
import time
from time import time as timer

import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'svg'

/home/cfchen/anaconda3/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
option = {
    'model': 'cfg/yolo.cfg',
    'load': 'bin/yolo.weights',
    'threshold': 0.2,
    'gpu': 0.85
}

tfnet = TFNet(option)

Parsing ./cfg/yolo.cfg
Parsing cfg/yolo.cfg
Loading bin/yolo.weights ...
Successfully identified 203934260 bytes
Finished in 0.12343406677246094s
Model has a coco model name, loading coco labels.

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
       |        | input                            | (?, 608, 608, 3)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 608, 608, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 304, 304, 32)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 304, 304, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | conv 1x1p0_1  +bnorm  leaky      | (?, 152, 152, 64)
 Load  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 152, 152, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 76, 76, 128)
 Load  |  Yep!

## Without Video Resizing

In [3]:
capture = cv2.VideoCapture('/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/VideosHPB/IMAG0011.mp4')
#capture = cv2.VideoCapture('../VideosHPB/IMAG0011.mp4')
colors = [tuple(255 * np.random.rand(3)) for i in range(5)]  #range(100)

In [4]:
for color in colors:
    print(color)

(171.7181218708245, 107.75584662600325, 21.708657975747958)
(127.91686447120738, 116.98542709961416, 90.31682983118924)
(164.43968301042682, 68.75017065079707, 9.99750674680425)
(213.01990308611033, 157.7654368379493, 0.12103582429826121)
(222.80646947706123, 190.3426931254668, 203.3725953957976)


In [5]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 30.0, (1920,1080))

In [6]:
print("Video Dim: {}  {}  {}".format(capture.get(3), 'X', capture.get(4)))

Video Dim: 1920.0  X  1080.0


In [7]:
stime = time.time()
start = timer()

count = 0
while(True):

    ret, frame = capture.read()

    if ret == False:
        break;
    else:
        count = count + 1

    # B & W image (using red channel only)
    #frame = np.stack((frame[:,:,0],frame[:,:,0],frame[:,:,0]),axis=2)
    results = tfnet.return_predict(frame)
    #print(results)

    if ret:
        for color, result in zip(colors, results):
            tl = (result['topleft']['x'], result['topleft']['y'])
            br = (result['bottomright']['x'], result['bottomright']['y'])
            brX = result['bottomright']['x']
            brY = result['bottomright']['y']
            label = result['label']
            confidence = result['confidence']
            confidence = round(confidence, 1)
            frame = cv2.rectangle(frame, tl, br, color, 7)
            #frame = cv2.putText(frame, label, tl, cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)
            #frame = cv2.putText(frame, label, (brX - 50, brY + 20), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            frame = cv2.putText(frame, label + ' ' + str(confidence), tl, cv2.FONT_HERSHEY_COMPLEX, 0.65, (0, 255, 0), 2)

        out.write(frame)
        cv2.imshow('frame', frame)
        #
        #print("Shape of Frame: {}".format(frame.shape))
        #print("Type of Frame:  {}".format(type(frame)))
        #print("Results:        {}".format(result))
        #print('FPS         {:.1f}'.format(1 / (time.time() - stime)))  # Frame per seconds
        #
        if (cv2.waitKey(1) & 0xFF == ord('q')) or (ret == False):
            break

print('++++++++ Video End ++++++++')
  
capture.release()
out.release()
cv2.destroyAllWindows()

print('\nElapsed time = ' + str(timer() - start) + ' s\n')

++++++++ Video End ++++++++

Elapsed time = 15.052112340927124 s



## With Video Resizing

In [13]:
capture = cv2.VideoCapture('/media/cfchen/956df7bc-562e-4f24-8339-fd0b67f98888/Downloaded/VideosHPB/IMAG0011.mp4')
#capture = cv2.VideoCapture('../VideosHPB/IMAG0011.mp4')
colors = [tuple(255 * np.random.rand(3)) for i in range(5)]  #range(100)

In [14]:
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi',fourcc, 30.0, (640,360))
#out = cv2.VideoWriter('output.avi',fourcc, 30.0, (576,324))

In [15]:
print("Video Dim: {}  {}  {}".format(capture.get(3), 'X', capture.get(4)))

Video Dim: 1920.0  X  1080.0


In [16]:
def rescale_frame(frame, percent=75):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [17]:
stime = time.time()
start = timer()

count = 0
while(True):

    ret, frame = capture.read()

    if ret == False:
        break;
    else:
        count = count + 1
    
    #frame = rescale_frame(frame, percent=30)
    frame = cv2.resize(frame, (640, 360)) 
    #frame = cv2.resize(frame, (576, 324)) 
    #print("Video Dim:  {}".format(frame.shape))

    # B & W image (using red channel only)
    #frame = np.stack((frame[:,:,0],frame[:,:,0],frame[:,:,0]),axis=2)
    results = tfnet.return_predict(frame)
    #print(results)

    if ret:
        for color, result in zip(colors, results):
            tl = (result['topleft']['x'], result['topleft']['y'])
            br = (result['bottomright']['x'], result['bottomright']['y'])
            brX = result['bottomright']['x']
            brY = result['bottomright']['y']
            label = result['label']
            confidence = result['confidence']
            confidence = round(confidence, 1)
            frame = cv2.rectangle(frame, tl, br, color, 7)
            #frame = cv2.putText(frame, label, tl, cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 0), 2)
            #frame = cv2.putText(frame, label, (brX - 50, brY + 20), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)
            frame = cv2.putText(frame, label + ' ' + str(confidence), tl, cv2.FONT_HERSHEY_COMPLEX, 0.65, (0, 255, 0), 2)

        out.write(frame)
        cv2.imshow('frame', frame)
        #
        #print("Shape of Frame: {}".format(frame.shape))
        #print("Type of Frame:  {}".format(type(frame)))
        #print("Results:        {}".format(result))
        #print('FPS         {:.1f}'.format(1 / (time.time() - stime)))  # Frame per seconds
        #
        if (cv2.waitKey(1) & 0xFF == ord('q')) or (ret == False):
            break

print('++++++++ Video End ++++++++')
  
capture.release()
out.release()
cv2.destroyAllWindows()

print('\nElapsed time = ' + str(timer() - start) + ' s\n')

++++++++ Video End ++++++++

Elapsed time = 10.741349220275879 s

